# Aggregate Data
This notebook combines the hourly data into daily data. The benifift of this is that the hourly data contains timing errors when merging the dataset. Keeping the data hourly would increase the number of samples, but also increase the number of bad samples.

We will also calculate the solar efficiency rather than having a actual output. This makes the data more readable and trainable for our model.

First we read in our data we collected.

In [134]:
import numpy as np
import pandas as pd

df = pd.read_csv("out.csv")

In [136]:
# make percent attained row
df["dhi_efficiency"] = df["DHI"]/df["Clearsky DHI"]
df["dni_efficiency"] = df["DNI"]/df["Clearsky DNI"]
df["ghi_efficiency"] = df["GHI"]/df["Clearsky GHI"]

## Aggregate hours into days
Next we are setting the index of our dataset to the `DATE` column. This will make it easier to group together rows by day.

In [135]:
import datetime as dt
df['DATE'] =  pd.to_datetime(df['DATE'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('DATE')

Here we are removing all hours of the day where there was no sunlight. To do this we use a little trick in our dataset. The efficiency values we created are based off the clearsky models which are directly correlated with sunlight. Thus when we preformed the division, non-sunlight hours had an efficiacy of NaN. We can exploit this by removing all columns that returned null, or the night hours.

In [ ]:
grouped = df[df['dhi_efficiency'].notnull()]  
grouped = grouped.groupby('STATION')

In [137]:

agg_intv = 'D'

daily_summary = pd.DataFrame()

daily_summary['latitude'] = grouped['latitude'].resample(agg_intv).last()
daily_summary['longitude'] = grouped['longitude'].resample(agg_intv).last()
daily_summary['elevation'] = grouped['elevation'].resample(agg_intv).last()

daily_summary['temperature'] = grouped.Temperature.resample(agg_intv).mean()
daily_summary['dew_point'] = grouped['Dew Point'].resample(agg_intv).mean()
daily_summary['relative_humidity'] = grouped['Relative Humidity'].resample(agg_intv).mean()
daily_summary['hourly_precipitation'] = grouped['HourlyPrecipitation'].resample(agg_intv).mean()
daily_summary['station_pressure'] = grouped.HourlyStationPressure.resample(agg_intv).mean()
daily_summary['relative_humidity'] = grouped['Relative Humidity'].resample(agg_intv).mean()
daily_summary['wind_direction'] = grouped['Wind Direction'].resample(agg_intv).mean()
daily_summary['wind_speed'] = grouped['Wind Speed'].resample(agg_intv).mean()
daily_summary['hourly_visibility'] = grouped['HourlyVisibility'].resample(agg_intv).mean()
daily_summary['cloud_cover'] = grouped['cloud_cover'].resample(agg_intv).mean()

daily_summary['cloud_type'] = grouped['cloud_type'].resample(agg_intv).apply(pd.array)
daily_summary['weather_type'] = grouped['weather_type'].resample(agg_intv).apply(pd.array)

daily_summary['dhi_efficiency'] = grouped['dhi_efficiency'].resample(agg_intv).mean()
daily_summary['dni_efficiency'] = grouped['dni_efficiency'].resample(agg_intv).mean()
daily_summary['ghi_efficiency'] = grouped['ghi_efficiency'].resample(agg_intv).mean()

In [138]:
import json

cloud_types = []
with open('cloud_lookup.json') as json_file:
    cloud_json = json.load(json_file).values()
    for x in cloud_json:
        cloud_types.append(x['cloud_str'])
    cloud_types = list(dict.fromkeys(cloud_types))
    
with open('weather_lookup_converter.json') as json_file:
    weather_types = list(json.load(json_file).keys())
    
    
# seperate cloud column    
cloud_sep = pd.DataFrame([x for x in daily_summary['cloud_type'].apply(lambda item: dict(map(lambda x: (x, np.count_nonzero(item.to_numpy() == x)), item))).values]).fillna(0)
cloud_sep = cloud_sep.append(pd.DataFrame(columns = cloud_types)).fillna(0.0)
cloud_sep = cloud_sep.loc[:, cloud_sep.columns.notnull()]
cloud_sep = cloud_sep.set_index(daily_summary.index)
daily_summary = daily_summary.join(cloud_sep)

# seperate weather column    
weather_sep = pd.DataFrame([x for x in daily_summary['weather_type'].apply(lambda item: dict(map(lambda x: (x, np.count_nonzero(item.to_numpy() == x)), item))).values]).fillna(0)
weather_sep = weather_sep.append(pd.DataFrame(columns = weather_types)).fillna(0.0)
weather_sep = weather_sep.loc[:, weather_sep.columns.notnull()]
weather_sep = weather_sep.set_index(daily_summary.index)
daily_summary = daily_summary.join(weather_sep)

pd.set_option('display.max_columns', None)
daily_summary

latitude  longitude  elevation  temperature  dew_point  \
STATION DATE                                                                 
121     2017-01-01  32.15000 -111.16700      737.0     9.555556   3.555556   
        2017-01-02  32.15000 -111.16700      737.0    11.888889   0.444444   
        2017-01-03  32.15000 -111.16700      737.0    16.111111  -0.444444   
        2017-01-04  32.15000 -111.16700      737.0    17.888889   1.111111   
        2017-01-05  32.15000 -111.16700      737.0    17.444444   4.111111   
...                      ...        ...        ...          ...        ...   
93987   2017-12-27  31.23611  -94.75444       87.8     4.700000   0.300000   
        2017-12-28  31.23611  -94.75444       87.8     4.000000  -1.600000   
        2017-12-29  31.23611  -94.75444       87.8     6.500000   0.700000   
        2017-12-30  31.23611  -94.75444       87.8    10.400000   8.600000   
        2017-12-31  31.23611  -94.75444       87.8     5.500000   0.500000   

                    relative_humidity  hourly_precipitation  station_pressure  \
STATION DATE                                                                    
121     2017-01-01          75.717778                   NaN         27.327778   
        2017-01-02          50.391111                   NaN         27.564444   
        2017-01-03          36.868889                   NaN         27.617778   
        2017-01-04          36.750000                   NaN         27.560000   
        2017-01-05          46.236667                   NaN         27.436667   
...                               ...                   ...               ...   
93987   2017-12-27          76.756000                 0.000         30.118000   
        2017-12-28          64.877000                 0.000         30.085000   
        2017-12-29          67.972000                 0.000         30.010000   
        2017-12-30          92.447000                 0.002         29.838000   
        2017-12-31          71.797000                 0.000         29.998000   

                    wind_direction  wind_speed  hourly_visibility  \
STATION DATE                                                        
121     2017-01-01      243.444444    4.755556             10.000   
        2017-01-02      206.288889    2.622222             10.000   
        2017-01-03      205.566667    1.566667             10.000   
        2017-01-04      243.244444    1.911111             10.000   
        2017-01-05      257.822222    4.155556             10.000   
...                            ...         ...                ...   
93987   2017-12-27       36.460000    0.480000              9.700   
        2017-12-28       52.320000    0.210000             10.000   
        2017-12-29      143.790000    0.210000             10.000   
        2017-12-30       48.930000    0.210000              6.375   
        2017-12-31       18.620000    0.620000              9.300   

                    cloud_cover  \
STATION DATE                      
121     2017-01-01     0.488889   
        2017-01-02     0.000000   
        2017-01-03     0.000000   
        2017-01-04     0.000000   
        2017-01-05     0.000000   
...                         ...   
93987   2017-12-27     0.800000   
        2017-12-28     0.800000   
        2017-12-29     0.800000   
        2017-12-30     0.800000   
        2017-12-31     0.780000   

                                                           cloud_type  \
STATION DATE                                                            
121     2017-01-01  [mostly_cloudy, mostly_clear, mostly_cloudy, m...   
        2017-01-02  [clear, clear, clear, clear, clear, clear, cle...   
        2017-01-03  [clear, clear, clear, clear, clear, clear, cle...   
        2017-01-04  [clear, clear, clear, clear, clear, clear, cle...   
        2017-01-05  [clear, clear, clear, clear, clear, clear, cle...   
...                                                               ...   
93987   2017-

In [139]:
daily_summary = daily_summary.reset_index(drop=False)
daily_summary = daily_summary.drop(columns=['cloud_type', 'weather_type', 'elevation', 'wind_direction'])
daily_summary = daily_summary.dropna()
daily_summary.to_parquet('solar_cleaned.parquet', engine='fastparquet', compression='GZIP')